<a href="https://colab.research.google.com/github/SJhawar1010/LLM/blob/main/Satyam_554_LLM_Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import pandas as pd
import tensorflow as tf
from transformers import MarianTokenizer, TFMarianMTModel
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
data=pd.read_csv('/content/HindiNLP.csv')
data.head()

,English,Hindi
0,"Hello, how are you?","नमस्ते, आप कैसे हैं?"
1,What is your name?,आपका क्या नाम है?
2,Where are you from?,आप कहाँ से हैं?
3,Nice to meet you.,आपसे मिलकर अच्छा लगा।
4,How old are you?,आपकी आयु कितनी है?


In [14]:
tokenizer1 = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model1 = TFMarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

tokenizer2 = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model2 = TFMarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-de")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/298M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-de.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [22]:
input1 = tokenizer1(data['English'].tolist(), truncation=True, padding=True, max_length=128)
target1 = tokenizer1(data['Hindi'].tolist(), truncation=True, padding=True, max_length=128)

In [23]:
input_ids1 = tf.convert_to_tensor(input1['input_ids'])
attention_masks1 = tf.convert_to_tensor(input1['attention_mask'])
decoder_input_ids1 = tf.convert_to_tensor(target1['input_ids'])

In [24]:
dataset1 = tf.data.Dataset.from_tensor_slices(({
    'input_ids': input_ids1,
    'attention_mask': attention_masks1,
    'decoder_input_ids': decoder_input_ids1
}, decoder_input_ids1))
dataset1 = dataset1.shuffle(1000).batch(16)

In [26]:
input2 = tokenizer2(data['English'].tolist(), truncation=True, padding=True, max_length=128)
target2 = tokenizer2(data['Hindi'].tolist(), truncation=True, padding=True, max_length=128)

In [27]:
input_ids2 = tf.convert_to_tensor(input2['input_ids'])
attention_masks2 = tf.convert_to_tensor(input2['attention_mask'])
decoder_input_ids2 = tf.convert_to_tensor(target2['input_ids'])

In [28]:
dataset2 = tf.data.Dataset.from_tensor_slices(({
    'input_ids': input_ids2,
    'attention_mask': attention_masks2,
    'decoder_input_ids': decoder_input_ids2
}, decoder_input_ids2))
dataset2 = dataset2.shuffle(1000).batch(16)

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model1.compile(optimizer=optimizer, loss=loss)
model1.fit(dataset1, epochs=3)

Epoch 1/3


In [ ]:
model2.compile(optimizer=optimizer, loss=loss)
model2.fit(dataset2, epochs=3)

In [ ]:
def get_predictions(model, tokenizer, sentences):
    predictions = []
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, return_tensors="tf")
        generated_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
        predictions.append(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
    return predictions

In [ ]:
predictions1 = get_predictions(model1, tokenizer1, data['English'])

In [ ]:
predictions2 = get_predictions(model2, tokenizer2, data['English'])

In [ ]:
accuracy1 = accuracy_score(data['Hindi'], predictions1)
accuracy2 = accuracy_score(data['Hindi'], predictions2)

In [ ]:
models = ['Model 1 (English-Hindi)', 'Model 2 (English-German)']
accuracies = [accuracy1, accuracy2]

plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=accuracies, palette='viridis')
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Comparison of Model Accuracies')
plt.show()